In [1]:
from models.gpt4all import m

In [2]:
m

In [3]:
m.open()

llama_model_load: ggml ctx size = 6065.35 MB
llama_model_load: memory_size =  2048.00 MB, n_mem = 65536
llama_model_load: loading model part 1/1 from '/Users/lorenzo/.nomic/gpt4all-lora-quantized.bin'
llama_model_load: .................................... done
llama_model_load: model size =  4017.27 MB / num tensors = 291


In [5]:
prompt = """Answer the question based on the context below.\
    Context: You are a marketing and customer relationship management assistant,\
    your task is to determine whether a given tweet is a \
    potential lead or not. Provide your analysis of the following social media post (tweet)\
    as a potential lead in the context of marketing and customer relationship management.\
    Consider the following factors in your analysis, but feel free to use additional\
    factors as well:\
    - Mentions of product or service offerings, calls to action, inquiries about pricing\
    or availability\
    - Keywords or phrases commonly associated with potential leads in the context of\
    marketing and CRM\
    - The tone and sentiment of the tweet\
    - The author's profile and engagement history on social media\
    - Any relevant contextual factors, such as recent product launches, industry events or\
    trends, or competitor activity.\
    Tweet: "that dream moisturizer is fire! 😩😍 that’s gone have to stay at the house with me."\
    Question: Is the above tweet a potential lead? Yes or No? Why?."""
out = m.prompt(prompt=prompt)
print(out)

1. The user has tweeted about their experience with an airline company's service. This could be either positive (potential lead) OR negative feedback (not a potential lead).
